In [1]:
import minsearch

In [2]:
import json

In [3]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [4]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
#Now we want to index it
index = minsearch.Index(
    text_fields=['question','text','section'],
    keyword_fields=['course']
)

In [7]:
q = 'the course has already started, can I still enroll?'

In [8]:
index.fit(documents)

In [9]:
boost = {'question': 3.0, 'section': 0.5}

result = index.search(
    query=q,
    boost_dict=boost,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    num_results=5
)

In [10]:
result

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [11]:
from openai import OpenAI

In [15]:
client = OpenAI()

In [17]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

In [26]:
response.choices[0].message.content

"Whether you can still enroll in a course that has already started depends on several factors, including the policies of the institution or organization offering the course, the type of course, and the timing. Here are some general steps you can take to find out:\n\n1. **Check the Course Enrollment Policies**: Look at the website or contact the registrar's office of the institution offering the course to see if they allow late enrollment.\n\n2. **Contact the Instructor**: Reach out to the course instructor directly to inquire about the possibility of enrolling late. They might be willing to accommodate you, especially if you show eagerness and are willing to catch up on missed work.\n\n3. **Review the Course Structure**: For some courses, particularly online or self-paced ones, enrolling late might not be an issue as you can proceed at your own pace.\n\n4. **Evaluate the Missed Content**: Consider how much content you've missed and whether you can realistically catch up without hinderi

In [28]:
#We want the LLM to put the document content in the context of the answer
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FQA database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [29]:
context = ""

for doc in result:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [32]:
prompt = prompt_template.format(question=q, context=context).strip()

In [33]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)

response.choices[0].message.content

'Yes, even if the course has already started, you are still eligible to enroll and submit the homeworks. However, be aware that there will be deadlines for turning in the final projects.'